In [2]:
%pylab
import numpy as np
import matplotlib.pyplot as plt
import cv2

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [10]:
image = cv2.imread('images/Tio.jpeg')
cv2.imshow('ImagenOriginal',image)

image.shape

(316, 176, 3)

In [4]:
Nx, Ny = 512,512
image = cv2.resize(image,(Nx,Ny))
cv2.imshow('ImagenOriginal',image)

In [5]:
# Escala de grises
image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
cv2.imshow('ImagenGrises',image_gray)
image_gray.shape

(512, 512)

In [6]:
image_gray_float = np.float64(image_gray)

In [21]:
image1 = cv2.cvtColor(image_gray, cv2.COLOR_GRAY2RGB)

cv2.imshow('imagen',image1)

In [9]:
rojo = image1.copy()

rojo[:,:,0] = 0
rojo[:,:,1] = 0

cv2.imshow('rojo',rojo)

In [23]:
azul = image1.copy()

azul[:,:,1] = 0
azul[:,:,2] = 0

cv2.imshow('azul',azul)

In [25]:
verde = image1.copy()

verde[:,:,0] = 0
verde[:,:,2] = 0

cv2.imshow('verde',verde)

In [28]:
# Obtener la transformada de Fourier

FFT = np.fft.fft2(image_gray_float)

FFT = np.fft.fftshift(FFT) # Centrar las frecuencias a 0

FFTA = np.abs(FFT) # Valores muy grandes de la FFT, mejor trabajar con LOG

FFTLog = np.log10(FFTA)

In [29]:
cv2.imshow('Espectro de Fourier', np.uint8(255*FFTLog/np.max(FFTLog)))

In [30]:
# La idea es diseñar un filtro

F1 = np.arange( -0.5*Nx + 1, 0.5*Nx + 1, 1)
F2 = np.arange( -0.5*Ny + 1, 0.5*Ny + 1, 1)

[X,Y] = np.meshgrid(F1,F2)

D = np.sqrt(X**2+Y**2)
D = D/np.max(D)

print(D)

[[0.99609375 0.99414254 0.99219519 ... 0.99414254 0.99609375 0.99804879]
 [0.99414254 0.9921875  0.9902363  ... 0.9921875  0.99414254 0.99610141]
 [0.99219519 0.9902363  0.98828125 ... 0.9902363  0.99219519 0.99415789]
 ...
 [0.99414254 0.9921875  0.9902363  ... 0.9921875  0.99414254 0.99610141]
 [0.99609375 0.99414254 0.99219519 ... 0.99414254 0.99609375 0.99804879]
 [0.99804879 0.99610141 0.99415789 ... 0.99610141 0.99804879 1.        ]]


In [52]:
# Definimos un radio de Corte

D0 = 0.2

Huv = np.zeros((Nx,Ny))

for i in range(Nx):
    for j in range(Ny):
        if(D[i,j] > D0):
            Huv[i,j] = 1

In [48]:
cv2.imshow('FiltroDeAlta',np.uint8(255*Huv))

In [53]:
# Usamos el teorema de convolucion

Guv = Huv*FFT

# Tomamos magnitud

GuvA = np.abs(Guv)

# Escalar

GuvA = np.uint8( 255*GuvA/np.max(GuvA) )

In [44]:
cv2.imshow('EspectroconFiltro', np.uint8(GuvA*255) )

In [54]:
# Ahora tomemos la transformada inversa

Gxy = np.fft.ifft2(Guv)
Gxy = np.abs(Gxy)

print(Gxy)

[[13.35525637  2.39432934  9.63497982 ... 19.1513997  15.53548638
   6.55281061]
 [12.85435365  1.07487334  1.86750608 ...  9.43300307  7.63438164
   2.19942211]
 [ 9.24196485  1.33002342  1.34021428 ...  2.40085332  1.46650033
   0.99161065]
 ...
 [19.50874422  7.40419179  2.79107571 ...  4.85782597  9.16844984
  24.58594128]
 [24.81081193 13.85374821  4.17730726 ...  5.78810675 13.09692698
  26.83377089]
 [19.23148918 20.67467389 10.66934186 ... 15.77491911 21.54929352
  28.29253958]]


In [60]:
cv2.imshow('ImagenConFiltro', np.uint8(Gxy))

# References

https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch04.html

# Bordes.

Lugares de la imagen donde hay un cambio brusco de la intensidad.

$$ dI(x,y) = \sqrt{ \bigg(\frac{dI(x,y)}{dx}\bigg)^{2} + \bigg(\frac{dI(x,y)}{dx}\bigg)^{2} }  $$

In [15]:
kernel1 = np.array([ -0.5,1,0.5 ])
kernel2 = np.array( [[ -1 ], [0], [1]]  )

In [16]:
X = cv2.filter2D(image, -1, kernel1)
Y = cv2.filter2D(image, -1, kernel2)

In [17]:
# Matriz de intensidad

Mxy = np.sqrt(X**2+Y**2)

Mxy = Mxy/np.max(Mxy)

In [34]:
# Aplicar un filtro
kernel1 = np.array([ -1,0.01,1 ])
kernel2 = np.array( [[ -2 ], [0.01], [2]]  )
X = cv2.filter2D(image, -1, kernel1)
Y = cv2.filter2D(image, -1, kernel2)
Mxy = np.sqrt(X**2+Y**2)

Mxy = Mxy/np.max(Mxy)
mask = np.where( Mxy > 0.5, 255, 0)

mask = np.uint8(mask)

cv2.imshow('Bordes',mask)

In [26]:
video = cv2.VideoCapture(0)

run = True

while run:
    
    # Lectura de imagenes
    ret, frame1 = video.read()
    frame = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)
    
    frame_float = frame.astype(float)
    
    kernel1 = np.array([ -1,0,1 ])
    kernel2 = np.array( [[ -1 ], [0], [1]]  )
    
    X = cv2.filter2D(frame_float, -1, kernel1)
    Y = cv2.filter2D(frame_float, -1, kernel2)
    
    
    Mxy = np.sqrt(X**2+Y**2)
    
    Mxy = Mxy/np.max(Mxy)
    
    mask = np.where( Mxy > 0.1, 255, 0)

    mask = np.uint8(mask)
    
    cv2.imshow('Bordes',mask)
    
    k = cv2.waitKey(1)&0xFF
    if(k == ord('p')):
        
        print('Exit')
        run = False
        break
    
video.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 